In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Dense
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2 as cv
from google.colab.patches import cv2_imshow
import pickle

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/Final Year Project  /Training Data/PapilaDB-PAPILA-9c67b80983805f0f886b068af800ef2b507e7dc0/'

In [ ]:
img_names = sorted(os.listdir(root + "FundusImages"))
images = np.empty((int(len(img_names)), 256, 256, 3),dtype=np.uint8)
done = 0
for i in img_names:
  image = (cv.imread(root + "FundusImagesResized/" + i))
  images[done] = image
  done +=1
  print(str(int(100* (done/488)))+'% done')

In [ ]:
img_names = sorted(os.listdir(root + "FundusImages"))
imagesFullSize = np.empty((int(len(img_names)), 1934,2576,3),dtype=np.uint8)
done = 0
for i in img_names:
  image = (cv.imread(root + "FundusImages/" + i))
  imagesFullSize[done] = image
  done +=1
  print(str(int(100* (done/488)))+'% done')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Final Year Project  /Models/Exstract4.h5')

In [ ]:
predicions = model.predict(images)
predicions = tf.one_hot(tf.argmax(predicions, axis=-1), depth=3)
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
predicions = cv.convertScaleAbs(predicions.numpy().clip(0, 1))

16/16 [==============================] - 19s 406ms/step


In [ ]:
contours_list = []
done = 0
for i in predicions:
  gray = cv.cvtColor(i*255, cv.COLOR_BGR2GRAY)
  ret, thresh = cv.threshold(gray, 90, 255, 0)

  contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

  contours = sorted(contours, key=cv.contourArea, reverse=True)

  contours_list.append(contours[0])

  done +=1


In [ ]:
contours_array = np.concatenate(contours_list)

In [ ]:
np.save(root+'ExpertsSegmentations/Contours.npy', contours_array)

In [ ]:
contours_array = np.load(root+'ExpertsSegmentations/Contours.npy', allow_pickle=True)

In [ ]:
ROI =np.loadtxt(root+'ExpertsSegmentations/Centers.csv', delimiter=',')

In [ ]:
ROI =np.loadtxt(root+'ExpertsSegmentations/Centers.csv', delimiter=',',dtype=np.uint8)
img_names = sorted(os.listdir(root + "FundusImages"))
done = 0
y = 2576/256
x = 1934/256

for i in img_names:
  image = (cv.imread(root + "FundusImages/" + i))
  crop_x = int(ROI[0][0]*x)
  crop_y = int(ROI[0][1]*y)
  crop_w, crop_h = 650, 650
  crop_img = image[crop_y-crop_h:crop_y, crop_x:crop_x+crop_w]
  resize_img = cv.resize(crop_img, (256, 256))
  cv.imwrite((root + "Cropped/" + i), resize_img)
  done +=1
  print(str(int(100* (done/488)))+'% done')